In [1]:
#imports
import pickle
import torch
from torch import nn
import torchvision
import torch.optim as optim

In [2]:
#project specific imports
from modules.utils import *
from modules.build_dataset import *

In [3]:
#OP_tsv: OpenProt 1.6
#Ens_trx: mart_export (ensembl 95)
#trx_fasta: transcripts fasta (ensembl 95)
#sorfs: list of ribo-seq orfs (mudge & al., 2022)
data = Data(
    OP_tsv = 'data/human-openprot-r1_6-refprots+altprots+isoforms-+uniprot2019_03_01.tsv', 
    Ens_trx = 'data/mart_export.txt',
    trx_fasta = 'data/ensembl95_trxps.fasta',
    sorfs = 'data/sorfs.xlsx'
)

In [4]:
#ensembl_trx = data.ensembl_trx()
#pickle.dump(ensembl_trx, open('data/ensembl_trx.pkl', 'wb'))
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))

In [5]:
#trx_orfs = data.trx_orfs(ensembl_trx)
#pickle.dump(trx_orfs, open('data/trx_orfs.pkl', 'wb'))
trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))

In [4]:
#dataset = data.dataset(ensembl_trx, trx_orfs)
#pickle.dump(dataset, open('data/dataset.pkl', 'wb'))
dataset = pickle.load(open('data/dataset.pkl', 'rb'))

In [6]:
len(dataset)

15965

In [7]:
#nc_dataset = data.nc_dataset(ensembl_trx, trx_orfs)
#pickle.dump(nc_dataset, open('data/nc_dataset.pkl', 'wb'))
#nc_dataset = pickle.load(open('data/nc_dataset.pkl', 'rb'))

In [8]:
#sorfs_dataset = data.sorfs_dataset(ensembl_trx)
#pickle.dump(sorfs_dataset, open('data/sorfs_dataset.pkl', 'wb'))

In [11]:
data.split_dataset(dataset)